<a href="https://colab.research.google.com/github/IvanaG1t/iPython-on-Colaboratory/blob/main/Py_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Napomene

Tutorijal pred Vama je namenjen učenju pythona i osnovnih kocepata u data science-u. Tema je odabrana u vreme kada je bila deo naše trenutne svakodnevice, i u nadi da će motivisati pojedince da koriste zvanične i proverene podatke čak i kada oni zahtevaju obradu. Za donošenje epidemioloških zaključaka je potrebno mnogo dublje znanje - unapred Vas molim da ne koristite površne zaključke stečene ovde u opravdavanju bilo kog ponašanja koje je u suprotnosti sa opštim preporukama Svetske zdravstvene organizacije (WHO) i nadležnih organa.\
Ukoliko želite da svoje znanje koristite za argumentaciju nekog stava, svakako to učinite uz davanje proverenih podataka, reproducibilne metodologije i jasnih rezultata. Što se tiče ovih rezultata, po mom mišljenju u njima nema ničeg kontraverznog ni neočekivanog. Ukoliko mi ne verujete, svakako nema potrebe ni da učite od mene, te ovaj kurs nije za Vas.\
Podaci ECDC kasne jedan dan u odnosu na one koje ćete videti u novinama istog dana, tj. broj slučajeva određenog dana je ustvari statistika prethodnog dana zaključno sa ponoći.\
Pitanja, sugestije i povratne informacije možete slati na ognjen011@gmail.com\
**Naročito bih voleo da mi pošaljete kvalitetne i pouzdane izvore informacija koje bih mogao da prikažem ovde, kako za Srbiju tako i za svet.** Uslov je samo da budu dostupne u fajlu bilo kog formata.
Tutorijal pred Vama je inspirisan ovim sjajnim [projektom](https://crockol.shinyapps.io/covid-19_didacticexperiment/#section-italian-case). 

dr Ognjen Milićević\
Katedra za medicinsku statistiku i informatiku\
Medicinski fakultet\
Univerzitet u Beogradu

# Način upotrebe

Ukoliko niste upoznati sa konceptom Notebook-a, svaki prozor sa kodom se može pokretati. Prepoznaćete ih po sivoj pozadini i znaku "Play" kada kliknete na njih. Možete ih pokretati i sa shift-Enter, i to Vam je dovoljno za prolaz kroz tutorijal, jer shift-Enter na ćelijama sa tekstom će samo nastaviti na sledeću ćeliju.\
Između ćelija koda je ubačen tekst, pa je kod koji je funkcionalna celina često rasparčan u delove. Stoga, **kod tutorijala se mora pokretati od početka do kraja da bi ste dobili ispravne output-e**. Ukoliko prekinete sa radom i vratite se kada je strana ponovo učitana a sve promene nestale, kliknite na prvu ćeliju i stiskajte shift-Enter dok ne dođete do dela do kog ste stigli.\
U pitanju je privremena kopija, tako da slobodno možete praviti izmene po želji. Čak je i poželjno da eksperimentišete sa svojim izmenama.

# Uvod

Većina skripti u pytonu počinje uvoženjem biblioteka potrebnih za rad. Iako se uvoženje može raditi bilo gde u kodu, dobra praksa je da se to radi na početku, a neki saveti kažu i po azbučnom redu - sve u cilju preglednosti.

In [ ]:
#@title
import datetime
import io
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats
from sklearn.linear_model import LinearRegression

Ovde vidimo pomešano nekoliko vrsta uvoza:


*   prost uvoz biblioteke (import datetime); sadržaje iz nje pozivamo isključivo sa datetime.*
*   uvoz biblioteke pod aliasom (import pandas as pd); sadržaje iz nje pozivamo isključivo sa pd.*
*   uvoz sadržaja iz biblioteke bez uvoza ostatka (from plotly.subplots import make_subplots); sada možemo koristiti make_subplots ali ne i plotly.subplots


## Učitavanje fajla

Sledeći korak je obično definisanje parametara i ulaznih podataka. Dobro je da je i ovo na vidljivom mestu pri početku, jer ukoliko želimo da izmenimo podatke sa kojima radimo ili neki od parametara koji fiksiramo u kodu, znamo odmah gde da tražimo.\
Podaci koji ćemo koristiti su zvanični podaci ECDC (*European Center for Disease Control*) i svakodnevno su dostupni za preuzimanje u vidu Excel fajla na sledećoj adresi:\
https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide \
Postoji više načina da ove podatke učitamo. Uobičajena korisnička upotreba je da kliknemo na link za preuzimanje u browseru čime će se fajl spustiti na lokalnu mašinu, nakon čega je možemo dalje otvoriti u željenom programu, gledati i obrađivati. Međutim, iako to sada ne deluje bitno - strana koju upravo gledate živi na internetu i stoga je lokalna mašina nešto dosta drugačije od kompjutera koji koristimo. Način na koji se "imitira" sistem fajlova nalik lokalnom se donekle razlikuje u zavisnosti od firme koja tu uslugu pruža (u ovom slučaju Google). Srećom, mi imamo lak način da zaobiđemo ovaj problem - biblioteka pandas može pročitati podatke direktno putem web adrese. Adresu saznajemo tako što umesto levog klika, desnim klikom i sa *Copy link address* dobijamo adresu samog fajla.\
Pošto se ime fajla menja sa datumom, i naš kod koristi interni datum da skine pravi fajl.

In [ ]:
#@title
today_date = datetime.datetime.strftime(datetime.datetime.today(), '%Y-%m-%d')
data_url = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{}.xlsx".format(today_date)
try:  # pokušaj skidanja današnjeg fajla
  data = pd.read_excel(data_url)
except:  # još nije postavljen današnji fajl, skinućemo jučerašnji
  print("Danasnja baza jos nije uploadovana. Skida se jucerasnja baza...")
  yesterday = datetime.datetime.strftime(datetime.datetime.today() - datetime.timedelta(days=1), '%Y-%m-%d')
  data_url = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{}.xlsx".format(yesterday)
  data = pd.read_excel(data_url)

Pandas uvek učitava svoje podatke u objekat koji se zove dataframe. Njega karakteriše prvenstveno organizacija - redovi su opservacije tj. jedinice posmatranja, dok su kolone promenljive koje pratimo. Ključna karakteristika je i da su kolone heterogene po tipu, dakle mešaju se numerički podaci sa kategoričkim.

## Pregled podataka

Najbolji način da proverimo uspešnost učitavanja i da bacimo prvi pogled na podatke kada radimo jeste da koristimo komandu *print*. Tako ćemo videti broj redova i imena nekih od kolona, ali većina redova i kolona će ostati neprikazana kod velikih dataframeova, indikovano tačkicama.

In [ ]:
#@title
print(data)

Jedna od prednosti ovog i njemu sličnih notebook-a je da imaju sjajan sistem za printanje koji se aktivira poslednjom komandom koja nema skladištenje u promenljivu (recimo da nema znak jednakosti), i nešto je lepši za dataframe-ove:


In [ ]:
#@title
data

Osim što smo dobili bojene redove i lepše formatiranu tabelu, dobili smo i sve kolone (što neće uvek biti slučaj, već su različiti limiti broja kolona koji se štampa, ali eto srećne okolnosti). Redovi se nisu promenili.\
Primetimo da su podaci i dalje malo nejasni, pogotovo što vidimo samo 10 redova sa jako malim brojem slučaja. Da bismo pregledali ostatak tabele, a da ne izlazimo i koristimo Excel, možemo koristiti neki od naprednih vizualizatora. Iskoristimo jednu od funkcija koje smo na početku uvezli iz IPython.display: 

In [ ]:
#@title
display(data)

Vidimo da je upravo funkcija display odgovorna za ulepšavanje našeg ispisa koje koristi notebook. Ako iskoristimo i HTML funkciju i malo se strpimo, možemo dobiti sve redove kroz koje možemo skrolovati: 

In [ ]:
#@title
display(HTML(data.to_html()))

Međutim, ovo je rizičan pristup. Broj redova će rasti, i biće potrebno sve više vremena i memorije za ovakav prikaz. U lokalu lako možete zablokirati sistem pokušajem štampanja velikih baza. Uostalom, dupli skrol često pravi probleme i daje neželjeno ponašanje (probajte točkić miša). Stoga, srednje rešenje je paginacija - štampanje određenog broja redova uz mogućnost menjanja strane. Ovakvi prikazi imaju aktivne elemente (dugmiće) i za kategoriju su složeniji od pasivnog teksta, ma koliko lepo formatiran:

In [ ]:
#@title
%load_ext google.colab.data_table
data

Komandu *%load_ext* pokrećete samo jednom. Pošto aktivni elementi zahtevaju spoljni jezik JavaScript (JS), rešenje zavisi od okruženja i gornje komande neće svuda raditi. Ukoliko volite klasični prikaz, možete se vratiti na staro sledećim komandama (u suprotnom, preskočite ih):

In [ ]:
#@title
%unload_ext google.colab.data_table
data

Za koji god da smo se pregled odlučili, vidimo da podaci imaju relativno jednostavnu strukturu: kolona dateRep je datum, zatim sledeće tri kolone predstavljaju dan mesec i godinu, tj. parsiran datum ukoliko je to lakše za upotrebu; broj zaraženih i broj smrti, i na kraju ime i šifra teritorije. Iako to ne piše, možemo primetiti da brojevi slučajeva nisu monotono rastući, i zaključujemo da **nije u pitanju ukupan broj slučajeva tog dana, već broj novoregistrovanih slučajeva. Isto važi i za smrtne slučajeve**.\
Skroz levo postoji kolona (koja ponekad ima ime a ponekad ne, zavisi od prikaza) koja se zove *indeks*. Ova kolona je po default-u samo broj reda počev od nule (zaglavlje se ne računa). Da li je to isto kao broj reda u Excel-u? Nije, to je zapravo IME tog reda, i umesto broja bi tu mogao stajati bilo koji string. Zamislite bazu pacijenata u kome bi indeksna kolona mogla biti JMBG. Ograničenje je da bi indeks trebalo da bude unikatan za svaki red.\
Zašto je nama ovo za sada bitno? Samo bi trebalo zapamtiti da se, kao ni pravo ime, ni broj reda ne menja. Čak i kada bismo izdvojili samo poslednjih par redova i snimili u novi dataframe, njihovi indeksi neće postati 0,1,2... već će ostati kakva su bila, što možda niste očekivali:

In [ ]:
#@title
data_last = data.iloc[6007:,]  # primetite da za slajsovanje koristimo iloc - obavezno u pandas-u
data_last

Za kraj ćemo reći da će naš fokus svakako biti numerički podaci, i da oni mogu biti celi brojevi (integer) ili decimalni (float) - za našu analizu ne pravimo razliku. Svi drugi tipovi promenljivih su za nas kategorički, i mada mogu biti konkretnog složenog tipa (kao gore viđen datum), mi ćemo ih ovde tretirati kao tekst (strings).\
Ovim završavamo pregled podataka i krećemo njihovu obradu.

# Deskripcija numeričkih podataka

Kada opisujemo podatke, pokušavamo da na jedan jednostavan način izvučemo glavne osobine. Nekada se misli na aritmetičku sredinu ili neku sličnu meru, međutim jasno je da u ovakvoj bazi ne možemo odmah da prikažemo prosek jer nije informativan. Naime, pomešane su države, ima puno dana sa nulama a opet ne počinju sve države od istog dana. Ukratko, podaci su strukturisani tako da prosek kolone nema informacioni značaj.\
Dobar početak je grafički prikaz podataka. Odmah ćemo reći da postoji više opcija za korišćenje biblioteka za grafike. Odlučili smo se za naprednu i lepu opciju (Plotly) umesto bazične (Matplotlib). Manjkavost je što menjanjem biblioteke menjamo i sintaksu koju koristimo, a veća je šansa da se zamene napredne biblioteke.\
Plotly ima svoju express i graphic_object varijantu, i zato smo dva puta importovali nešto iz plotly-ja (pogledajte u prvoj ćeliji). Prvo ćemo koristiti jednostavne ekspres funkcionalnosti, a onda ćemo pokušati da napravimo neke napredne vizualizacije.

## Bazični grafici

In [ ]:
#@title
fig = px.scatter(data, x="cases", y="deaths")
fig.show()

Primetimo da lebdenjem kursorom iznad tačaka dobijamo tačne vrednosti. Zatim primetimo izdvojene tačke sa desne strane - ekstremene i usamljene vrednosti uvek treba dodatno ispitati. U trenutku pisanja ovo ga postoja la je usamljena tačka od 15141 slučajeva, koju će kasnije prestići druge velike države. Međutim, sa grafika ne možemo dobiti državu iz koje tačka potiče, već možemo naći tačku sa tom vrednošću u dataframe-u. Takođe možemo da iskoristimo činjenicu da nema mnogo tačaka sa x-vrednošću preko 15000. Ili možemo da tražimo da vidimo najveću vrednost X-ose (mada će se ova vrednost menjati).

In [ ]:
#@title
print(data.loc[data["cases"]==15141,])  # tačno 15141 slučajeva
print(data.loc[data["cases"]>14000,])  # više od 14000
print(data.loc[(data["cases"]>14000) & (data["cases"]<16000),])  # između 14000 i 16000
print(data.loc[data["cases"]==max(data["cases"]),])  # maksimalno slučajeva

Sasvim očekivano, država u pitanju je Kina. Da smo dobili neku neočekivanu državu iz konteksta pandemije COVID-19, trebalo bi proveriti kod i/ili podatke.\
Ovo je dobar trenutak da prokomentarišemo .loc koje se pojavljuje u kodu. Ukoliko ste koristili MATLAB ili R, znate da pri pristupanju dvodimenzionalnim podacima (dakle tabelama i matricama) dovoljno je da date red i kolonu razdvojene zarezom, baš kao i ovde, ali tamo nije potrebna nikakva dodatna reč. Međutim, ako ovde pokušate bez .loc dobićete neobičnu grešku. Razlog za ovo je što biblioteka Pandas pokušava da imitira korišćenje SQL baze, u kojima se indeks može sastojati iz više kolona, te je to i ovde slučaj. Kada ne koristite .loc ili sličnu odrednicu, program misli da ste mu dodelili indeks od dve kolone, a ne red i kolonu. Na početku se ovo .loc često zaboravlja ali u nekom trenutku ulazi u prste.\
Podatke ne moramo prikazivati takozvanim 2D vizualizacijama (gde poredimo dve dimenzije, ovde slučajeve i smrti) već ih možemo pojedinačno prikazivati. Iako nije mnogo poznat, najbolji brzi predgled se dobija [box-plot](https://en.wikipedia.org/wiki/Box_plot)-om:

In [ ]:
#@title
fig1 = px.box(data, y="cases")
fig1.show()
fig2 = px.box(data, y="deaths")
fig2.show()

Primetićemo da naši box plotovi uopšte ne liči na kutiju koju očekujemo, i takođe su nam poprilično beskorisni. Razlog za to je što je većina vrednosti oko nule. Možemo da probamo da "zumiramo" tako što odaberemo samo ograničen interval vrednosti blizu nule:

In [ ]:
#@title
fig2 = px.box(data, y="deaths")
fig2.update_yaxes(range=[0, 10])  # opseg prikazanih vrednosti na y-osi
fig2.show()

I dalje nismo ništa rešili, jer nisu problem vrednosti blizu nule, već upravo nule. Da li imamo prava da ih isključimo? Zamislite da je neko počeo da vodi ovu evidenciju pre 10 godina, i svaki dan dodaje broj slučajeva pandemije korona virusa. Pošto iste nije bilo do sada, svaki dan bi se pisale nule za sve države. Ali ove vrednosti su trivijalne, i mi ih nismo ni uzeli u obzir. Istom logikom možemo isključiti i sve nule u trenutnoj tabeli.\
(Ovo nije potpuno tačno i treba paziti kada ima smisla koristiti pojednostavljujuće pretpostavke. Trenutno nule znače da je država prijavila nulu, što znači da monitoring postoji. Kada se ne radi monitoring, mi možemo reći da vrednosti nema (prazno polje u Excel-u, ili zavismo od programskog jezika jedna od {NA, NAN, NULL}. Samim tim, nekada će nam nule biti važne, ali ne i pri deskripciji.)

In [ ]:
#@title
fig1 = px.box(data.loc[data["cases"]>0,:], y="cases")
fig1.show()
fig2 = px.box(data.loc[data["deaths"]>0,:], y="deaths")
fig2.show()

Sada se već nazire kutija, ali opet je grafik ružan. Možemo pokušati da prikažemo podatke transformisane logaritmovanjem, što je česta praksa sa podacima koji su nagomilani na jednu stranu (ovde ka malim vrednostima). I dalje isključujemo nule jer logaritam od nule je minus beskonačno.

In [ ]:
#@title
fig1 = px.box(data.loc[data["cases"]>0,:], y="cases", log_y=True)
fig1.show()
fig2 = px.box(data.loc[data["deaths"]>0,:], y="deaths", log_y=True)
fig2.show()

Primetite da nismo logaritmovali same podatke, već 
smo prikazali y-osu u logaritamskoj razmeri. To je mogućnost ovog paketa i nije uvek dostupna u analizi. Prikaz logaritmovanih podataka je nezgodan za čitaoca jer brojevi nemaju smisla dok se ne vrate u originalni oblik.

## Napredni grafici

Ako smo rekli da koristimo naprednu biblioteku, želimo to i da pokažemo naprednim funkcionalnostima. Lako možemo dodati sadržaj na već postojeće grafike.\
Za početak, možemo iskombinovati scatter plot sa box plotovima i dobiti sjajnu informaciju. Radi estetike, prikazaćemo samo vrednosti gde je i broj obolelih i umrlih veči od nule, i to u logaritamskoj razmeri.

In [ ]:
#@title
data_positive = data.loc[(data["cases"]>0) & (data["deaths"]>0),:]  # zagrade su jako bitne!!
fig = px.scatter(data_positive, x="cases", y="deaths", marginal_y="box",
           marginal_x="box", log_x=True, log_y=True)
fig.show()

Možemo na originalni scatter plot dodati regresionu pravu, tj. liniju trenda.

In [ ]:
#@title
data_positive = data.loc[(data["cases"]>0) & (data["deaths"]>0),:]  # zagrade su jako bitne!!
fig = px.scatter(data_positive, x="cases", y="deaths", trendline="ols")
fig.show()

Ovo je dobar trenutak da popričamo o regresiji i ekstremnim vrednostima na nivou koji su potrebni za osnovnu analizu podataka. Prvo, da bismo uopšte smeli da koristimo regresiju podaci se ne smeju levkasto širiti ili skupljati oko linije, kao što je ovde slučaj. Ovaj formalizam se često ignoriše jer iako formalno pogrešna, analiza može biti informativna. Međutim, često ćete videti zloupotrebu pogrešno predviđenih trendova (jedan primer je loše obavljena regresija ljudi koji negiraju klimatske promene - periodični podaci zahtevaju složenu analizu a ne običnu liniju). Ako ćete na taj način zloupotrebljavati ovu tehniku, nemojte reći da ste je ovde naučili :) Ovde nećemo insistirati na levkastom širenju, već ćemo primetiti da iako ova linija izgleda adekvatno za veće vrednosti, ne znamo koliko dobro opisuje male vrednosti koje su i najčešće. Da bismo bolje pogledali, možemo probati logaritamski prikaz (mada prava više neće biti prava).

In [ ]:
#@title
data_positive = data.loc[(data["cases"]>0) & (data["deaths"]>0),:]  # zagrade su jako bitne!!
fig = px.scatter(data_positive, x="cases", y="deaths", trendline="ols", log_x=True, log_y=True)
fig.show()

Iako ova linija ne izgleda loše, problem je što uopšte ne opisuje dobro male podatke - vidimo da linija prolazi iznad vrednosti u donjem levom uglu, a ne kroz njih kako bi trebalo. Ovo smo i došli da pogledamo na logaritmovanom plotu. Intuicija bi trebalo da Vam kaže da je deo problema ona jedna ekstremna vrednost koja vuče pravu ka sebi. Pokušajmo da je izbacimo i ponovimo crtanje linije.

In [ ]:
#@title
data_positive = data.loc[(data["cases"]>0) & (data["deaths"]>0) & (data["cases"]<12000),:]  # zagrade su jako bitne!!
fig = px.scatter(data_positive, x="cases", y="deaths", trendline="ols", log_x=True, log_y=True)
fig.show()

Ako uporedite dve linije, videćete drastičnu razliku izazvanu samo malim udelom tačaka. Vrednosti sa jako niskim brojem slučajeva u svakom slučaju ne možemo dobro modelovati (počeci ili krajevi epidemija).
Podsećamo da se kvalitet fita nije video na originalnom (nelogaritmovanom) grafiku. Dakle, potreban je oprez i gledanje podataka se zaista ne može zameniti.

# Stratifikacija podataka

Ova krupna reč zapravo znači analizu podataka po grupama (stratumima). Zbog jednostavnosti naših podataka, jedina grupišuća promenljiva je država. Bar još dve grupišuće promenljive bi bile jako važne za COVID-19 - starosna grupa i zdravstveno stanje. Međutim, za sada ćemo se fokusirati na raspodelu po državama, ali imajte u vidu da koliko god da imate promenljivih u svojoj bazi, uvek postoje takozvane "skrivene" promenljive koje utiču na podatke.

## Prikaz po grupi (državi)

Pošto je grupišuća varijabla nominalna, nema potrebe stavljati je na osu - tj. poredak država nije bitan. Ovaj tip podataka se najbolje prikazuje stubičastim dijagramom (bar plot). Pogledajmo jučerašnji broj obolelih po državi:

In [ ]:
#@title
danas = datetime.datetime.today() 
juce = danas - datetime.timedelta(days=1)  # ne mozemo samo oduzeti jedan jer python ne zna sta je ta jedinica

# Dva nacina da dobijemo jucerasnje podatke
# 1) uporedimo i mesec i dan i godinu
jucerasnji_podaci = data.loc[(data["year"]==juce.year) & (data["month"]==juce.month) & (data["day"]==juce.day),:]

# 2) da formatiramo datum i poredimo samo njega
juce_datum = datetime.datetime.strftime(juce, '%m-%d-%Y')
jucerasnji_podaci = data.loc[data["dateRep"]==juce_datum,:]

Nemojte da Vas zbunjuje ovo ponavljanje reči "datetime", to je naprosto način na koji su developeri strukturirali paket. Za one koji su radili objektno orijentisano programiranje, u paketu datetime postoji klasa datetime čije metode koristimo.

In [ ]:
#@title
# Prikažimo podatke
fig = px.bar(jucerasnji_podaci, x='geoId', y='cases')
fig.show()

Primetimo da oznake deluju potpuno pogrešno - najveću vrednost ima JM (Jamaica). Pošto poznajemo kontekst podataka, znamo da to nije istina. Ako se bolje zagledamo, videćemo da su oznake šire od samih stubića. Probajmo da ih smanjimo.


In [ ]:
#@title
# Prikažimo podatke
sz = 5
fig = px.bar(jucerasnji_podaci, x='geoId', y='cases')
fig.update_xaxes(tickfont=dict(family='Arial', color='blue', size=sz))
fig.show()

Slova su dosta sitna, ali vidimo sve oznake. Probajte da povećate atribut size i videćete da će se opet izgubiti pojedine države. Takođe, kako vreme prolazi i pojavljuju se nove države možda ćemo morati još da smanjimo slova. U principu, retko crtamo ovoliko grupa. Takođe, nacrtali smo samo jedan dan, i stiče se utisak da smo mogli da bolje prikažemo informacije.\
Metodološki preuzeto iz SQL baza podataka - ukoliko želimo da prikažemo operacije na dataframe-u grupisanom po nekoj promenljivoj, koristimo funkciju *groupby*(). Ova operacija deli dataframe na listu dataframe-ova, po jedan za svaku grupu, i osim što ih možemo jedan po jedan procesirati, možemo i koristiti neku od čestih, takozvanih agregujućih funkcija. To su funkcije tipa mean, max, min, sum, count itd. koje proizvode jednu vrednost od više redova i stoga se mogu upisati u novu tabelu koja ima jedan red po grupi. Na primer, ukupan broj slučajeva je svakako interesantan.

In [ ]:
#@title
sums1 = data.groupby("geoId")["cases"].sum()
sums2 = data.groupby("geoId")[["cases"]].sum()
sums3 = data.groupby("geoId", as_index=False)[["cases"]].sum()
display(sums1)
display(sums2)
display(sums3)

Date su tri moguće komande za sumiranje po grupama. Prva varijanta (sums1) izdvaja samo jednu kolonu, te dataframe postaje serija (Series) koja se printa kao običan tekst. Serije su gradivne jedinice dataframe-a. U drugoj komandi (sums2) smo duplirali uglaste zagrade, čime je reč "cases" postala lista ["cases"], i time signaliziramo da želimo listu kolona a ne samo jednu, te analogno rezultat ostaje dataframe od samo jedne kolone. Primetimo da su se u prva dva slučaja indeksi promenili i zovu se kao države po kojima smo grupisali, što je inače sasvim ok. Pošto plotly preferira da sve promenljive za crtanje budu u pravim kolonama, mi menjamo parametar as_index da bude False i time dobijamo pravu kolonu sa državama koju ćemo koristiti za crtanje.

In [ ]:
#@title
fig = px.bar(sums3, x="geoId", y='cases')
fig.update_xaxes(tickfont=dict(family='Arial', color='blue', size=sz))
fig.show()

Da nismo svaki put birali kolonu "cases" sve vrednosti bi se sabirale, uključujući i datume. Mi uvek dodatno biramo kolone koje želimo da uđu u narednu operaciju. Ukoliko bismo želeli grubu procenu smrtnosti (bez ulaženja u činjenicu da ne znamo ukupan broj testiranih) kao broj umrlih kroz broj slučajeva, moramo da: 

1.   izdvojimo dve kolone
2.   saberemo ih po datumima
3.   podelimo i stavimo u novu kolonu po imenu "Mort"
4.   prikažemo bar plotom



In [ ]:
#@title
sums = data.groupby("geoId", as_index=False)[["cases","deaths"]].sum()
sums["Mort"] = sums["deaths"]/sums["cases"]
fig = px.bar(sums, x="geoId", y='Mort')
fig.update_xaxes(tickfont=dict(family='Arial', color='blue', size=sz))
fig.show()

Neophodno je i ovde se ograditi da je smrtnost populaciona mera, i moramo znati o kojoj populaciji je reč. Mi radimo na populaciji koja ima pozitivan test na COVID-19. Ekstenzija ovih brojki na opštu populaciju nije moguća.

## Vremenske serije

Iako je vreme kontinuirana promenljiva, izdvajamo posebnu grupu gde je vreme na x-osi. Značaj ove grupe je što obuhvata mnoge probleme iz različitih disciplina (ekonomske, biološke, telekomunikacione...) a metodologija obrade je u velikoj meri zajednička. Za nas je značajan problem količine podataka - umesto jedne agregovane vrednosti, mi sada imamo nizove vrednosti promenljivih u vremenu. Svaki ovaj niz se naziva vremenska serija, pa umesto jedne vrednosti za svaku grupu sada imamo jednu vremensku seriju po grupi, što je izazovnije za prikaz i obradu.\
Klasičan prikaz vremenskih serija je linijski dijagram. Pogledajmo oboljevanje u vremenu u Švajcarskoj:

In [ ]:
#@title
fig = px.line(data.loc[data["geoId"]=="CH"], x="dateRep", y='cases')
fig.show()

Vidimo da je danas (22.3.2020) u toku drugi skok u Švajcarskoj od početka pandemije. Ovo se ne može videti iz jedne agregovane vrednosti, koja god ona bila. Ovo je značaj vremenskih serija. Međutim, nećemo valjda ručno gledati svih 175 država? Nećemo, ali nemamo pametan način da ih sve pregledamo. Pogledaćemo nekoliko reprezentativnih slučajeva u nadi da ćemo dobiti ideju kako da dalje obrađujemo podatke.\
Pogledajmo pet država sa najviše slučajeva ukupno:

In [ ]:
#@title
sorted_series = sums.sort_values(by="cases", ascending=False)
top_5 = sorted_series["geoId"][:5]
data_top_5 = data.loc[data["geoId"].isin(top_5),:]
fig = px.line(data_top_5, x="dateRep", y="cases", color="geoId")
fig.show()

Do sada smo se sve vreme držali plotly express paketa, koji je jednostavniji ali i manje moćan od ostatka ove biblioteke. Neke stvari za sada možemo uraditi samo uz pomoć naprednih funkcija u *plotly.graph_objects*, kao na primer sledeći primer - kada posmatramo dve promenljive koje su povezane ali je jedna višestruka veća od druge, teško se prikazuju na istom grafiku jer se manja "zalepi" za x-osu. Rešenje je korišćenje grafika sa dve y-ose:

In [ ]:
#@title
data_china = data.loc[data["geoId"]=="CN",:]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=data_china["dateRep"], y=data_china["cases"], name="cases"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=data_china["dateRep"], y=data_china["deaths"], name="deaths"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Chinese Cases and Deaths"
)

# Set x-axis title
fig.update_xaxes(title_text="Datum")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Cases", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Deaths", secondary_y=True)

fig.show()

Da li bismo mogli da nacrtamo sve države na istom grafiku? Mogli bismo pokušati, ali to može zahtevati puno vremena ili memorije, naročito kada je grafik interaktivan (kao što naši jesu) ili su u pitanju vektorski formati slika (PDF, SVG, EPS). Čak i da je taj problem rešen, u gomili linija ne bismo mogli da razaberemo one od interesa, a opseg vrednosti je takav da se države sa malim brojem slučaja ne bi ni videle od Kine.

# Proširivanje podataka

Pošto su podaci koje smo do sada koristili poprilično jednostavni, možemo da pokušamo da nađemo dodatne baze. Recimo da nas zanima obim testiranja, s tim što moramo u startu biti svesni da je ova informacija mnogo teža za pronalaženje u etru jer, osim što je teža za praćenje, ima puno potencijalnih implikacija na kvalitet i zdravstvenu politiku.\
Ukoliko odemo na sajt [OurWorldInData](https://ourworldindata.org/covid-testing) i kliknemo na tab Data ispod interaktivnog scatter plota pod imenom "Tests conducted vs. Total confirmed cases of COVID-19", biće nam ponuđeno da skinemo CSV sa podacima. Primetimo da sada ne možemo dobiti direktni link za preuzimanje kao pre (probajte dobijeni link u drugom prozoru), pa ćemo morati da ga preuzmemo na lokalnu mašinu iz pretraživača. Nakon toga, možemo pogledati koje su nam opcije za upload u sledećem [tutorijalu](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=eikfzi8ZT_rW). Ovaj kod je specifičan za ovu platformu i podložan je promenama sa razvojem.


In [ ]:
#@title
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Gornji kod je preuzet iz tutorijala, iako ste uspešno uploadovali adekvatan fajl (i to samo jedan), njegovo ime je i dalje u promenljivoj fn. Na žalost, sadržaj fajla je uvezen kao niz bajtova pa ćemo se poslužiti dodatnim paketom io da bismo dobili pandas dataframe.

In [ ]:
#@title
tests_raw_df = pd.read_csv(io.BytesIO(uploaded[fn]))
tests_raw_df

Ova tabela je značajno lošije sređena od prošle, i to nam je svakako jedan od indikatora da bi podaci mogli biti manje pouzdani. Ukupan broj slučajeva nije tako ažurno vođen, broj testiranja je retko prisutan, postoji mešanje država i okruga, promanljiva Year je poprilično nenintuitivna. Uz to, skraćenice država su troslovne a u drugoj tabeli dvoslovne, te ni u tom smislu nemamo sreće. Ovo je realan problem dve tabele iz različitih izvora.\
Predlažem da iz tabele sa testiranjima izvučemo jedan red po državi koji ima i broj testova i broj slučajeva. Ako neka država nema testove preskočićemo je. Iako deluje da svaka država ima samo jednom upisan broj testova, ukoliko ih ima više unapred odlučujemo da ćemo uzeti najveći tj. poslednji u vremenu (jer sve vrednosti su kumulativne) korišćenjem funkcije max. Na kraju ćemo izračunati novu promenljivu - proporciju pozitivnih testova.

In [ ]:
#@title
# Prvo cemo naci indekse svih redova gde su maksimumi svojih drzava.
# Sta bi se desilo da ima vise istih vrednosti? Da li bi se odabrala prva?
# Nasumicna? Svaka od njih?
idx = tests_raw_df.groupby(['Entity'])['Total COVID-19 tests'].transform(max) == tests_raw_df['Total COVID-19 tests']
tests_max_df = tests_raw_df[idx]
# Racunamo procenat pozitivnih testova
tests_max_df["Pos_ratio"] = tests_max_df['Total confirmed cases of COVID-19 (cases)']/tests_max_df['Total COVID-19 tests']
tests_max_df

Primetimo da se broj redova značajno smanjio, i da neke vrednosti fale. Pošto su nedostajuće vrednosti mahom broj slučajeva, što mi posedujemo u drugoj tabeli, mogli bismo da te vrednosti popunimo, ali to prevazilazi obim ovog kursa.\
Najbolje je spojiti dobijenu tabelu sa tabelom suma koju smo ranije dobili, pošto obe tabele predstavljaju agregovane statistike za niz dana (da je tabela sa testiranjem bolje popunjena možda bismo mogli da povežemo i pune tabele, ali avaj). Da bismo to uradili, potrebna nam je kolona kooja postoji u ove tabele. Na žalost, već smo primetili da skraćenica koju smo do sada koristili nije u istom formatu, pa ćemo ponovo generisati sume iz prve tabele, ovaj put agregovane po punim imenima.\
Spajanje se vrši funkcijom merge, koja je analogna [JOIN](https://www.edureka.co/blog/sql-joins-types) operaciji kod baza podataka. Pošto želimo samo redove koji postoje u obe tabele, koristićemo unutrašnji (inner) join.

In [ ]:
#@title
sums_full_name = data.groupby("Countries and territories", as_index=False)[["cases","deaths"]].sum()
sums_full_name["Mort"] = sums_full_name["deaths"]/sums_full_name["cases"]
merged_inner = pd.merge(left=sums_full_name, right=tests_max_df, how='inner', left_on='Countries and territories', right_on='Entity')
merged_inner

Odmah možemo i pogledati naše izvedene promenljive, udeo smrtnosti i udeo pozitivnih na testu. Funkcije za crtanje nemaju problem sa nedostajućim vrednostima.

In [ ]:
#@title
fig = px.scatter(merged_inner, x="Mort", y="Pos_ratio", log_x=True, log_y=True)
fig.show()

Napomena: *u trenutku pisanja, tačka sa najmanjim udelom pozitivnih na testu je bila Australija. Nakon provere, ispostavilo se da je broj testova nekih 127000 dok su u tom periodu novine objavljivale cifru od 30000 testova. Nezavisno od toga šta je istina, pouka je da uvek gledamo ekstremne vrednosti i proveravamo ih za greške.*

Recimo da hoćemo da pogledamo korelaciju između ove dve izvedene promenljive. Koristićemo corrcoef funkciju za Pirsonovu korelaciju iz paketa numpy, mada dolazi u obzir i Spirmanova korelacija koja je u scipy.stats paketu. Pre toga moramo ukloniti prazne vrednosti iz tabele.

In [ ]:
#@title
clean = merged_inner.loc[~(merged_inner['Pos_ratio'].isnull() | merged_inner['Mort'].isnull())]
print("Pirsonova korelacija:")
print(np.corrcoef(clean['Mort'], clean['Pos_ratio']))
print("Spirmanova korelacija:")
scipy.stats.spearmanr(clean['Mort'], clean['Pos_ratio'])

Nemojte da Vas zbuni korelaciona matrica koja se dobija za Pirsonov koeficijent korelacije - na dijagonali će uvek biti jedinice, dok vrednost koja nas zanima je na sporednoj dijagonali.\
U zavisnosti od znaka, intenziteta i statističke značajnosti, tumačimo koliko su ove dve pojave korelisane. Pozitivan koeficijent znači da je korelacija pozitivna (kada raste jedna raste i druga), veličina apsolutne vrednosti ide do 1 i govori o jačini povezanosti, dok se p-vrednosti manje od 0.05 arbitrarno smatraju statistički značajnim.\
U trenutku pisanja pozitivna srednje jaka korelacija od oko 0.5 indikuje da ove promenljive zajedno rastu, što možemo objasniti na više načina. Očigledan bi bio da veći obim testiranja nalazi manje pozitivnih slučajeva ali i veći udeo lakših, te je i smrtnost u takvoj grupi manja. Sa druge strane, kada testiramo samo bolesnike sa jakim simptomima više njih će biti pozitivno ali je i smrtnost očekivano veća.

# Linearna regresija

Python je dosta loš za linearnu regresiju sa osnovim paketima u poređenju sa drugim data science jezicima - output je siromašniji, a sama procedura je komplikovanija jer je prilagođena prediktivnom modelovanju mašinskim učenjem. Ipak, prikazaćemo osnovne korake ovde.\
Linearna regresija daje sličnu informaciju kao regresija, ali nam daje i matematički linearni model koji možemo koristiti za predikciju vrednosti - za određenu vrednost promenljive X mi ćemo znati očekivanu vrednost promenljive Y. Pokušaćemo da nađemo ovaj model za izvedene promenljive korišćene u prethodnom poglavlju - udeo umrlih i udeo pozitivnih.

In [ ]:
#@title
# Nalaženje modela
regressor = LinearRegression()  
regressor.fit(clean[['Mort']], clean['Pos_ratio'])  # primetite duple zagrade oko X-promenljive

# Računanje R2 (mora posebno)
r2 = regressor.score(clean[['Mort']], clean['Pos_ratio'])

print("Koeficijent: {}".format(regressor.coef_))
print("Intercept: {}".format(regressor.intercept_))
print("Koeficijent: {}".format(regressor.coef_))
print("R2: {}".format(r2))

R2 ili koeficijent determinacije predstavlja udeo objašnjene varijacije u modelu, a zove se tako jer je jednak kvadratu Pirsonovog r koje smo računali u prošlom poglavlju.

# Italijanski slučaj

Italijanski slučaj je značajan ne samo kao velika tragedija, već i kao važan predmet izučavanja za epidemiologiju. Pogledaćemo neke osnovne podatke koje Protezione Civile objavljuje na svom Github-u.

In [ ]:
#@title
dataITALY = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv")
dataREGIONI = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv")
dataPROVINCE = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv")
dataITALY

In [ ]:
#@title
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=dataITALY["data"], y=dataITALY["totale_attualmente_positivi"], name="Cases"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=dataITALY["data"], y=dataITALY["terapia_intensiva"], name="Intensive Care"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Italian Cases, Regular and Difficult"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Total", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Intensive", secondary_y=True)

fig.show()